In [1]:
import sys
sys.path.append('../..')
from model import processing
from model.amm.money_market import MoneyMarket, MoneyMarketAsset, CDP
from model.amm.trade_strategies import liquidate_cdps
mm = processing.get_money_market()
print(mm)

Pool Address: 0x1b02E051683b5cfaC5929C25E84adb26ECf87B38
money_market(    liquidity: {'USDC': 389203855524, 'USDT': 348352132976, 'WBTC': 1083220017, 'DOT': 6788758377384185, 'vDOT': 3307933100023583}
    liquidation threshold: {('USDC', 'USDT'): 0.93, ('USDC', 'WBTC'): 0.9, ('USDC', 'DOT'): 0.9, ('USDC', 'vDOT'): 0.9, ('USDT', 'USDC'): 0.93, ('USDT', 'WBTC'): 0.9, ('USDT', 'DOT'): 0.9, ('USDT', 'vDOT'): 0.9, ('WBTC', 'USDC'): 0.7, ('WBTC', 'USDT'): 0.7, ('WBTC', 'DOT'): 0.7, ('WBTC', 'vDOT'): 0.7, ('DOT', 'USDC'): 0.8, ('DOT', 'USDT'): 0.8, ('DOT', 'WBTC'): 0.8, ('DOT', 'vDOT'): 0.9, ('vDOT', 'USDC'): 0.7, ('vDOT', 'USDT'): 0.7, ('vDOT', 'WBTC'): 0.7, ('vDOT', 'DOT'): 0.9}
    liquidation bonus: {('USDC', 'USDT'): 0.014999999999999902, ('USDC', 'WBTC'): 0.030000000000000027, ('USDC', 'DOT'): 0.030000000000000027, ('USDC', 'vDOT'): 0.030000000000000027, ('USDT', 'USDC'): 0.014999999999999902, ('USDT', 'WBTC'): 0.030000000000000027, ('USDT', 'DOT'): 0.030000000000000027, ('USDT', 'vDOT'

In [6]:
from model.amm.money_market import CDP
# replace DOT collateral with HDX
hdx = MoneyMarketAsset(
    name='HDX',
    price=0.01,
    liquidity=100000000,
    liquidation_bonus=0.07,
    liquidation_threshold=0.7,
    ltv=0.6
)
mm.add_new_asset(hdx)
# distribution = processing.distribute_value(
#     num_positions=50,
#     total_value=1000000,
#     concentration=0.8
# )
price_ratio = mm.get_oracle_price('DOT') / mm.get_oracle_price('HDX')
hdx_cpds = []
for cdp in mm.cdps:
    if 'DOT' in cdp.collateral:
        hdx_collateral = cdp.collateral['DOT'] * price_ratio * 8 / 7
        other_collateral = {tkn: cdp.collateral[tkn] for tkn in cdp.collateral if tkn != 'DOT'}
        new_cdp = CDP(
            collateral={'HDX': hdx_collateral, **other_collateral},
            debt=cdp.debt.copy()
        )
        precision = 20
        new_cdp.health_factor = mm.get_health_factor(new_cdp)
        if abs(cdp.health_factor - new_cdp.health_factor) / cdp.health_factor > 2 ** -precision:
            for i in range(precision):
                # print(f"target: {cdp.health_factor}, value: {new_cdp.health_factor}")
                if mm.get_health_factor(new_cdp) < cdp.health_factor:
                    new_cdp.collateral['HDX'] += hdx_collateral * 2 ** -i
                else:
                    new_cdp.collateral['HDX'] -= hdx_collateral * 2 ** -i
                new_cdp.health_factor = mm.get_health_factor(new_cdp)
        
        hdx_cdps.append(new_cdp)

{('USDC', 'USDT'): 0.93, ('USDC', 'WBTC'): 0.9, ('USDC', 'DOT'): 0.9, ('USDC', 'vDOT'): 0.9, ('USDT', 'USDC'): 0.93, ('USDT', 'WBTC'): 0.9, ('USDT', 'DOT'): 0.9, ('USDT', 'vDOT'): 0.9, ('WBTC', 'USDC'): 0.7, ('WBTC', 'USDT'): 0.7, ('WBTC', 'DOT'): 0.7, ('WBTC', 'vDOT'): 0.7, ('DOT', 'USDC'): 0.8, ('DOT', 'USDT'): 0.8, ('DOT', 'WBTC'): 0.8, ('DOT', 'vDOT'): 0.9, ('vDOT', 'USDC'): 0.7, ('vDOT', 'USDT'): 0.7, ('vDOT', 'WBTC'): 0.7, ('vDOT', 'DOT'): 0.9, ('USDC', 'HDX'): 0.9, ('HDX', 'USDC'): 0.7, ('USDT', 'HDX'): 0.9, ('HDX', 'USDT'): 0.7, ('WBTC', 'HDX'): 0.0, ('HDX', 'WBTC'): 0, ('DOT', 'HDX'): 0.8, ('HDX', 'DOT'): 0.7, ('vDOT', 'HDX'): 0.7, ('HDX', 'vDOT'): 0.7}


In [7]:
omnipool = processing.get_omnipool()
# print(omnipool)

Omnipool: omnipool
********************************
tvl cap: inf
lrna fee:

    4-Pool: 0.0005
    KSM: 0.0005
    HDX: 0.0005
    KILT: 0.0005
    WETH: 0.0005
    2-Pool101: 0.0005
    GLMR: 0.0005
    iBTC11: 0.0005
    BNC: 0.0005
    WBTC19: 0.0005
    RING: 0.0005
    vASTR: 0.0005
    vDOT: 0.0005
    CFG: 0.0025
    CRU: 0.0005
    2-Pool102: 0.0005
    DOT: 0.0005
    AAVE: 0.0005
    PHA: 0.0005
    tBTC: 0.0005
    ZTG: 0.0005
    INTR: 0.0005
    ASTR: 0.0005
    SOL: 0.0005

asset fee:

    4-Pool: 0.0015
    KSM: 0.002285
    HDX: 0.0015
    KILT: 0.0015
    WETH: 0.0015
    2-Pool101: 0.0015
    GLMR: 0.0015
    iBTC11: 0.0015
    BNC: 0.0015
    WBTC19: 0.0015
    RING: 0.0015
    vASTR: 0.0015
    vDOT: 0.0015
    CFG: 0.0015
    CRU: 0.0015
    2-Pool102: 0.0015
    DOT: 0.0015
    AAVE: 0.0015
    PHA: 0.0015
    tBTC: 0.0015
    ZTG: 0.0015
    INTR: 0.0015
    ASTR: 0.0015
    SOL: 0.0015

asset pools: (

    *4-Pool*
    asset quantity: 1204249.2002105224
    lrna

In [11]:
from model.amm.trade_strategies import liquidate_cdps
from model.amm.global_state import GlobalState
from model.amm.agents import Agent

initial_state = GlobalState(
    money_market=mm,
    pools={'omnipool': omnipool},
    agents={
        'liquidator': Agent(
            enforce_holdings=False,
            trade_strategy=liquidate_cdps()
        )
    }
)

In [ ]:
time_steps = 100
initial_price = 
prices = [{'HDX': }]

In [ ]:
def print_position(data):
    print('Hydration Account: ', data['account'])
    print('availableBorrowsBase: ', data['availableBorrowsBase'])
    print('borrower: ', data['borrower'])
    print('config:')
    for tkn, tkn_cfg in data['config']['assets'].items():
        print(f"    {tkn}:")
        # print("        underlyingAsset: ", tkn_cfg["underlyingAsset"])
        print("        balance: ", tkn_cfg["balance"])
        print("        debt: ", tkn_cfg["debt"])

    print('currentLiquidationThreshold', data['currentLiquidationThreshold'] / 100)
    print('healthFactor', data['healthFactor'])
    print('calculated health factor', data['calculatedHealthFactor'])
    print('discrepancy:', data['discrepancy'])
    print('ltv', data['ltv'])
    # print('pool', data['pool'])
    print('totalCollateralBase', data['totalCollateralBase'])
    print('totalDebtBase', data['totalDebtBase'])
    print('updated', data['updated'])
    print()
    print('---------------------------------------------------')

def simple_print(position):
    assets = list(filter(lambda tkn: position['config']['assets'][tkn]['balance'] > 0 or position['config']['assets'][tkn]['debt'] > 0, position['config']['assets'].keys()))
    # print(assets)
    print(f"position {position['account']}:")
    for tkn in assets:
        print(
            f"    {tkn}:\n"
            f"        balance: {position['config']['assets'][tkn]['balance']}\n"
            f"        debt: {position['config']['assets'][tkn]['debt']}"
        )
    print(f"    liquidation threshold: {position['currentLiquidationThreshold']}")
    print(f"    (calculated): {position['calculatedLiquidationThreshold'] * 100}")
    # print(f"    health factor: {position['healthFactor']}")
    print('---------------------------------------------------')

total_positions = len(borrower_data)
correct_positions = 0

if borrower_data:
    # try to correctly calculate health factor
    for i, position in enumerate(borrower_data):
        if position['healthFactor'] == 0:
            print('skipping position with 0 health factor')
            position['discrepancy'] = 0
            continue
            
        tokens = position['config']['assets']
        collateral_tokens = [
            {
                'name': t,
                **reserves_data[t],
                **tokens[t]
            } for t in filter(lambda k: tokens[k]['balance'] > 0, tokens.keys())
        ]
        debt_tokens = [
            {
                'name': t,
                **reserves_data[t],
                **tokens[t]
            } for t in filter(lambda k: tokens[k]['debt'] > 0, tokens.keys())
        ]

        liquidation_threshold = calculate_liquidation_threshold(position['config']['assets'])
        health_factor = (
            sum([
                mpf(c_tkn['balance']) * c_tkn['priceInMarketReferenceCurrency']  # * c_tkn['liquidityIndex']  # / 10 ** c_tkn['decimals']
                for c_tkn in collateral_tokens
            ]) * liquidation_threshold
            / sum([
                mpf(d_tkn['debt']) * d_tkn['priceInMarketReferenceCurrency']  # * d_tkn['variableBorrowIndex']  # / 10 ** d_tkn['decimals']
                for d_tkn in debt_tokens
            ])
        )
        position['calculatedHealthFactor'] = health_factor
        position['discrepancy'] = 1 - health_factor / position['healthFactor']
        position['calculatedLiquidationThreshold'] = liquidation_threshold
        
        if abs(position['discrepancy']) > 0.00001:
            print(f"position ({i}) didn't work out:")
            # print_position(position)
            print(f"calculated health factor: {health_factor} vs actual: {position['healthFactor']} discrepancy = {round(position['discrepancy'], 12)}")
            if len(collateral_tokens) > 1:
                print(f"{len(collateral_tokens)} collateral tokens.")
            if len(debt_tokens) > 1:
                print(f"{len(debt_tokens)} debt tokens.")
            # break;
        else:
            print(f"position{i} successfully calculated ({position['calculatedHealthFactor']} ~= {position['healthFactor']}) discrepancy ={round(position['discrepancy'], 12)}")
            correct_positions += 1

print(f"{round(correct_positions / total_positions * 100)}% correct")
# collateral.scaledATokenBalance * collateral.priceInMarketReferenceCurrency * collateral.liquidityIndex * collateral.eModeLiquidationThreshold
# --------------------------------divided by------------------------------------------
# debt_token.scaledVariableDebt * debt_token.priceInMarketReferenceCurrency * debt_token.variableBorrowIndex

In [ ]:
for position in [borrower_data[k] for k in [5, 6, 7, 8, 9]]:
    simple_print(position)

In [ ]:
# look at the collateral and debt assets of all positions whose health factor wasn't calculated correctly
c2 = 0
dot_positions = list(filter(lambda position: position['config']['assets']['Polkadot']['balance'] > 0, borrower_data))
for i, position in enumerate(borrower_data):
    if position in dot_positions:
        tokens = position['config']['assets']
        collateral_tokens = [t for t in filter(lambda k: tokens[k]['balance'] > 0, tokens.keys())]
        debt_tokens = [t for t in filter(lambda k: tokens[k]['debt'] > 0, tokens.keys())]
        if len(collateral_tokens) > 1:
            print(i, collateral_tokens, debt_tokens)
            c2 += 1
print (c2)
print(len(dot_positions))

In [ ]:
# simlplified reserves data
reserves_data = {
    "USDC": {
        "priceInMarketReferenceCurrency": 0.99986185,
        "reserveLiquidationThreshold": 0.9,
        "eModeLabel": "Stablecoins",
        "eModeLiquidationThreshold": 0.93
    },
    "Tether USD": {
        "priceInMarketReferenceCurrency": 0.99984867,
        "reserveLiquidationThreshold": 0.9,
        "eModeLabel": "Stablecoins",
        "eModeLiquidationThreshold": 0.93
    },
    "Bitcoin (Moonbeam Wormhole)": {
        "priceInMarketReferenceCurrency": 82944.62880389,
        "reserveLiquidationThreshold": 0.7,
        "eModeLabel": "",
        "eModeLiquidationThreshold": 0.0
    },
    "Polkadot": {
        "priceInMarketReferenceCurrency": 4.08314409,
        "reserveLiquidationThreshold": 0.8,
        "eModeLabel": "DOT",
        "eModeLiquidationThreshold": 0.9
    },
    "Bifrost Voucher DOT": {
        "priceInMarketReferenceCurrency": 5.94003953,
        "reserveLiquidationThreshold": 0.7,
        "eModeLabel": "DOT",
        "eModeLiquidationThreshold": 0.9
    }
}

In [ ]:
print_position(borrower_data[252])

In [ ]:
import sys
sys.path.append('../..')
from model.processing import get_omnipool_router
router = get_omnipool_router()
omnipool = router.exchanges['omnipool']

In [ ]:
print(omnipool)

In [ ]:
from model.amm.money_market import MoneyMarket, CDP
money_market = MoneyMarket(
    liquidity = {asset['name']: asset['availableLiquidity'] / 10 ** asset['decimals'] for asset in reserves_data.values()},
    oracle = omnipool.oracles['price'],
    liquidation_threshold = {
        (asset1['name'], asset2['name']): (
            asset1['eModeLiquidationThreshold'] 
            if asset1['eModeLabel'] == asset2['eModeLabel']
            else asset1['reserveLiquidationThreshold']
        )
        for asset1 in reserves_data.values()
        for asset2 in reserves_data.values() if asset2 != asset1
    },
    min_ltv = {asset['name']: asset['baseLTVasCollateral'] for asset in reserves_data.values()},
    liquidation_penalty= 0.01,
    cdps = None,
    full_liquidation_threshold = None,
    partial_liquidation_pct = 0.5
)

In [ ]:
len([
    position for position in borrower_data 
    if len([
        1 for tkn in position['config']['assets'] 
        if position['config']['assets'][tkn]['usageAsCollateralEnabledOnUser'] and not position['config']['assets'][tkn]['scaledATokenBalance'] > 0
    ]) > 0
])

In [ ]:
def calculate_liquidation_threshold(position):
    """
    Calculate the liquidation threshold for a position with multiple assets.
    
    Args:
        position: Dictionary with asset balances and debts
        reserves_data: Dictionary with asset price and threshold data
    
    Returns:
        float: The calculated liquidation threshold as a decimal (0.0-1.0)
    """
    # Step 1: Calculate all asset values in USD
    collateral_values = {}
    debt_values = {}
    total_collateral_value = 0
    total_debt_value = 0
    
    # Group by eMode
    collateral_by_emode = {}
    debt_by_emode = {}
    
    # Track which assets are in which eMode
    assets_by_emode = {}
    
    for asset_name, asset_data in position.items():
        if asset_name == "liquidation threshold" or asset_name == "calculated":
            continue
            
        balance = float(asset_data.get("balance", 0))
        debt = float(asset_data.get("debt", 0))
        price = reserves_data[asset_name]["priceInMarketReferenceCurrency"]
        emode_label = reserves_data[asset_name]["eModeLabel"]
        
        # Initialize eMode group if needed
        if emode_label not in assets_by_emode:
            assets_by_emode[emode_label] = []
        
        if emode_label and asset_name not in assets_by_emode[emode_label]:
            assets_by_emode[emode_label].append(asset_name)
        
        # Process collateral
        if balance > 0:
            value = balance * price
            collateral_values[asset_name] = {"amount": balance, "value": value}
            total_collateral_value += value
            
            if emode_label not in collateral_by_emode:
                collateral_by_emode[emode_label] = 0
            collateral_by_emode[emode_label] += value
        
        # Process debt
        if debt > 0:
            value = debt * price
            debt_values[asset_name] = {"amount": debt, "value": value}
            total_debt_value += value
            
            if emode_label not in debt_by_emode:
                debt_by_emode[emode_label] = 0
            debt_by_emode[emode_label] += value
    
    # Check for self-borrowing (same asset used as both collateral and debt)
    self_borrowing_assets = [
        asset for asset in position 
        if asset != "liquidation threshold" and asset != "calculated" and 
        float(position[asset].get("balance", 0)) > 0 and float(position[asset].get("debt", 0)) > 0
    ]
    
    if self_borrowing_assets:
        # For self-borrowing, use the reserve threshold instead of eMode
        return reserves_data[self_borrowing_assets[0]]["reserveLiquidationThreshold"]
    
    # Check for special Bifrost DOT case with small position size or near 1:1 ratio
    if "Bifrost Voucher DOT" in position and float(position["Bifrost Voucher DOT"].get("balance", 0)) > 0:
        bifrost_value = float(position["Bifrost Voucher DOT"]["balance"]) * reserves_data["Bifrost Voucher DOT"]["priceInMarketReferenceCurrency"]
        
        if "Polkadot" in position and float(position["Polkadot"].get("debt", 0)) > 0:
            polkadot_debt_value = float(position["Polkadot"]["debt"]) * reserves_data["Polkadot"]["priceInMarketReferenceCurrency"]
            
            # If it's a small position or ratio close to 1:1, use reserve threshold
            if polkadot_debt_value < 1000 or abs(bifrost_value / polkadot_debt_value - 1.0) < 0.15:
                return reserves_data["Bifrost Voucher DOT"]["reserveLiquidationThreshold"]
    
    # Start the matching process and build a list of (value, threshold) pairs
    matches = []
    remaining_collateral = {k: {"amount": v["amount"], "value": v["value"]} for k, v in collateral_values.items()}
    remaining_debt = {k: {"amount": v["amount"], "value": v["value"]} for k, v in debt_values.items()}
    
    # Step 1: First match debt assets with collateral in the same eMode group
    for emode, assets in assets_by_emode.items():
        if not emode:  # Skip empty eMode label
            continue
        
        # Get debt and collateral in this eMode
        debt_assets_in_mode = [asset for asset in assets if asset in remaining_debt]
        collateral_assets_in_mode = [asset for asset in assets if asset in remaining_collateral]
        
        if not debt_assets_in_mode or not collateral_assets_in_mode:
            continue  # No matching possible in this eMode
        
        # Calculate total values in this eMode
        total_debt_in_mode = sum(remaining_debt[asset]["value"] for asset in debt_assets_in_mode)
        total_collateral_in_mode = sum(remaining_collateral[asset]["value"] for asset in collateral_assets_in_mode)
        
        if total_debt_in_mode <= 0 or total_collateral_in_mode <= 0:
            continue
        
        # If there's enough collateral to cover all debt in this eMode, use eMode threshold for all
        if total_collateral_in_mode >= total_debt_in_mode:
            emode_threshold = reserves_data[assets[0]]["eModeLiquidationThreshold"]
            
            matches.append({
                "value": total_debt_in_mode,
                "threshold": emode_threshold,
                "type": f"eMode match: {emode}"
            })
            
            # Reduce collateral assets proportionally by the amount of debt they cover
            collateral_reduction_ratio = total_debt_in_mode / total_collateral_in_mode
            for asset in collateral_assets_in_mode:
                reduction = remaining_collateral[asset]["value"] * collateral_reduction_ratio
                remaining_collateral[asset]["value"] -= reduction
            
            # Remove all debt in this eMode as it's fully matched
            for asset in debt_assets_in_mode:
                remaining_debt[asset]["value"] = 0
        else:
            # If not enough collateral, match what we can and leave rest for next step
            emode_threshold = reserves_data[assets[0]]["eModeLiquidationThreshold"]
            
            matches.append({
                "value": total_collateral_in_mode,
                "threshold": emode_threshold,
                "type": f"eMode match (partial): {emode}"
            })
            
            # Use all available collateral in this eMode
            for asset in collateral_assets_in_mode:
                remaining_collateral[asset]["value"] = 0
            
            # Reduce debt proportionally
            debt_reduction_ratio = total_collateral_in_mode / total_debt_in_mode
            for asset in debt_assets_in_mode:
                reduction = remaining_debt[asset]["value"] * debt_reduction_ratio
                remaining_debt[asset]["value"] -= reduction
    
    # Step 2: Match remaining debt with collateral using reserve thresholds
    # For optimal matching, we'll sort collateral by threshold (highest first)
    remaining_collateral_assets = sorted(
        [asset for asset in remaining_collateral if remaining_collateral[asset]["value"] > 0],
        key=lambda asset: reserves_data[asset]["reserveLiquidationThreshold"],
        reverse=True
    )
    
    remaining_debt_assets = [
        asset for asset in remaining_debt if remaining_debt[asset]["value"] > 0
    ]
    
    # Process each debt asset
    for debt_asset in remaining_debt_assets:
        debt_value = remaining_debt[debt_asset]["value"]
        if debt_value <= 0:
            continue
            
        debt_remaining = debt_value
        
        # Try to match this debt with the best available collateral
        for collateral_asset in remaining_collateral_assets:
            collateral_value = remaining_collateral[collateral_asset]["value"]
            if collateral_value <= 0:
                continue
                
            match_amount = min(collateral_value, debt_remaining)
            if match_amount > 0:
                matches.append({
                    "value": match_amount,
                    "threshold": reserves_data[collateral_asset]["reserveLiquidationThreshold"],
                    "type": f"reserve match: {debt_asset} -> {collateral_asset}"
                })
                
                remaining_collateral[collateral_asset]["value"] -= match_amount
                debt_remaining -= match_amount
                
                if debt_remaining <= 0:
                    break
    
    # # Step 3: Add any unused collateral with its reserve threshold
    # # This should only happen if there's excess collateral beyond what's needed for debt
    # remaining_collateral_assets = [
    #     asset for asset in remaining_collateral 
    #     if remaining_collateral[asset]["value"] > 0
    # ]
    
    # for asset in remaining_collateral_assets:
    #     remaining_value = remaining_collateral[asset]["value"]
    #     if remaining_value > 0:
    #         matches.append({
    #             "value": remaining_value,
    #             "threshold": reserves_data[asset]["reserveLiquidationThreshold"],
    #             "type": f"unused collateral: {asset}"
    #         })
    
    # Calculate weighted average of thresholds
    total_matched_value = sum(match["value"] for match in matches)
    weighted_sum = sum(match["value"] * match["threshold"] for match in matches)
    
    if total_matched_value == 0:
        return 0  # Edge case: no collateral
        
    weighted_threshold = weighted_sum / total_matched_value
    
    # Round to 2 decimal places (if needed)
    return weighted_threshold

In [ ]:
calculate_overall_liquidation_threshold(borrower_data[252])

In [ ]:
[asset_symbol for asset_symbol in reserves_data]

In [ ]:
borrower_data[252]['config']['assets']['USDC']

In [ ]:
reserves_data = {
    "USDC": {
        "priceInMarketReferenceCurrency": 0.99986185,
        "reserveLiquidationThreshold": 0.9,
        "eModeLabel": "Stablecoins",
        "eModeLiquidationThreshold": 0.93
    },
    "Tether USD": {
        "priceInMarketReferenceCurrency": 0.99984867,
        "reserveLiquidationThreshold": 0.9,
        "eModeLabel": "Stablecoins",
        "eModeLiquidationThreshold": 0.93
    },
    "Bitcoin (Moonbeam Wormhole)": {
        "priceInMarketReferenceCurrency": 82944.62880389,
        "reserveLiquidationThreshold": 0.7,
        "eModeLabel": "",
        "eModeLiquidationThreshold": 0.0
    },
    "Polkadot": {
        "priceInMarketReferenceCurrency": 4.08314409,
        "reserveLiquidationThreshold": 0.8,
        "eModeLabel": "DOT",
        "eModeLiquidationThreshold": 0.9
    },
    "Bifrost Voucher DOT": {
        "priceInMarketReferenceCurrency": 5.94003953,
        "reserveLiquidationThreshold": 0.7,
        "eModeLabel": "DOT",
        "eModeLiquidationThreshold": 0.9
    }
}

# Borrower position data (replace with your actual data)
borrower_position = {
    "config": {
        "assets": {
            "USDC": {"balance": 129.96863071216722419575702264743008402550765284152, "debt": 1102.8428918536412668099246081827086285312810126042},
            "Tether USD": {"balance": 0.0, "debt": 0.0},
            "Bitcoin (Moonbeam Wormhole)": {"balance": 0.0, "debt": 0.0},
            "Polkadot": {"balance": 525.10397876476448949641808913491940222317909388275, "debt": 0.0},
            "Bifrost Voucher DOT": {"balance": 0.0, "debt": 0.0},
        }
    },
    currentLiquidationThreshold: 0.8074
}